<a href="https://colab.research.google.com/github/aayrm5/PySpark/blob/main/Working_with_RDD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Working with RDD (Resilient Distributed Dataset)**

**`Udemy Course: Best Hands-on Big Data Practices and Use Cases using PySpark`**

**`Author: Amin Karami (PhD, FHEA)`**

---

**Resilient Distributed Dataset (RDD)**: RDD is the fundamental data structure of Spark. It is fault-tolerant (resilient) and immutable distributed collections of any type of objects.

source: https://spark.apache.org/docs/latest/rdd-programming-guide.html

source: https://spark.apache.org/docs/latest/api/python/reference/

In [1]:
########## ONLY in Colab ##########
!pip3 install pyspark
########## ONLY in Colab ##########

     |████████████████████████████████| 281.4 MB 36 kB/s 
     |████████████████████████████████| 198 kB 64.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=cd48323a8adbc75a3a303b06c8d31420383ee293175aa62dbc6fa6f0c7ea765c
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
########## ONLY in Ubuntu Machine ##########
# Load Spark engine
!pip3 install -q findspark
import findspark
findspark.init()
########## ONLY in Ubuntu Machine ##########

In [3]:
# Linking with Spark
from pyspark import SparkContext, SparkConf

In [5]:
# Initializing Spark
conf = SparkConf().setAppName("RDD_practice").setMaster("local[*]")
sc = SparkContext(conf=conf)
print(sc)

<SparkContext master=local[*] appName=RDD_practice>


In [6]:
sc.defaultParallelism

2

# **Part 1: Create RDDs and Basic Operations**
# **There are two ways to create RDDs:**

1.   Parallelizing an existing collection in your driver program
2.   Referencing a dataset in an external storage system, such as a shared filesystem, HDFS, HBase, or any data source offering a Hadoop InputFormat.

In [ ]:
# Generate random data:


In [ ]:
# Create RDD:


In [ ]:
# Data distribution in partitions:


In [ ]:
# Print last partition


In [ ]:
# count():


In [ ]:
# first():


In [ ]:
# top():


In [ ]:
# distinct():


In [ ]:
# map():


In [ ]:
# filter(): 


In [ ]:
# flatMap():


In [ ]:
# Descriptive statistics:


In [ ]:
# mapPartitions():


# **Part 2: Advanced RDD Transformations and Actions**

In [ ]:
# union():


In [ ]:
# intersection():


In [ ]:
# Find empty partitions


In [ ]:
# coalesce(numPartitions):


In [ ]:
# takeSample(withReplacement, num, [seed])


In [ ]:
# takeOrdered(n, [ordering])


In [ ]:
# reduce():


In [ ]:
# reduceByKey():


In [ ]:
# sortByKey():


In [ ]:
# countByKey()


In [ ]:
# groupByKey():


In [ ]:
# lookup(key):


In [ ]:
# cache:
# By default, each transformed RDD may be recomputed each time you run an action on it.
# However, you may also persist an RDD in memory using the persist (or cache) method,
# in which case Spark will keep the elements around on the cluster for much faster access the next time you query it.


In [ ]:
# Persistence (https://spark.apache.org/docs/latest/rdd-programming-guide.html#rdd-persistence)
